In [1]:
import mpg.ml.model as mpg_model
import mpg.ml.model.mlp as mlp
import mpg.ml.model.gnn as gnn
import mpg.gnn.aggregator as agg
import mpg.gnn.gcn as gcn
import mpg.ml.model.utils as model_utils
import importlib
import tensorflow as tf
importlib.reload(mlp)
importlib.reload(gnn)
importlib.reload(gcn)
importlib.reload(agg)
importlib.reload(model_utils)

2023-06-17 02:56:21.975035: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 02:56:22.569361: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<module 'mpg.ml.model.utils' from '/home/ramizouari/Academic/AI/MeanPayOffGames/notebooks/mpg/ml/model/utils.py'>

In [92]:
n=20
m=5
p=0.3
batches=16
A=tf.cast(tf.random.uniform([batches,n,n]) > p,dtype=tf.float32)
W=tf.random.uniform([batches,n,n])
G=tf.stack([A,W],axis=-1)
X=tf.random.uniform([batches])
P=tf.math.softmax(tf.random.uniform([batches,n]),axis=-1)
V=tf.math.sign(tf.random.uniform([batches])-0.5)


A1=tf.cast(tf.random.uniform([n,n]) > p,dtype=tf.float32)
W1=tf.random.uniform([n,n])
G1=tf.stack([A1,W1],axis=-1)
P1=tf.math.softmax(tf.random.uniform([n]),axis=-1)
V1=tf.math.sign(tf.random.uniform([])-0.5)

A2=tf.cast(tf.random.uniform([m,m]) > p,dtype=tf.float32)
W2=tf.random.uniform([m,m])
G2=tf.stack([A2,W2],axis=-1)
P2=tf.math.softmax(tf.random.uniform([m]),axis=-1)
V2=tf.math.sign(tf.random.uniform([])-0.5)

A_s=tf.ragged.stack([A1,A2])
W_s=tf.ragged.stack([W1,W2])
G_s=tf.ragged.stack([G1,G2])
X_s=tf.constant([2,1])
V_s=tf.stack([V1,V2])
P_s=tf.RaggedTensor.from_tensor(tf.ragged.stack([P1,P2]).to_tensor(),[n,m])

model=mlp.MLP(graph_size=n,num_actions=n,state_shape=())
gnn_model=gnn.GNN(ragged_batch=True,masked_softmax=True)

2023-06-17 00:06:51.157120: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?]
	 [[{{node Placeholder}}]]


(None, None, None)
(None, 1, None)


2023-06-17 00:06:51.533544: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype int64 and shape [?]
	 [[{{node Placeholder}}]]


In [86]:
P_s.row_splits

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([ 0, 20, 25])>

In [87]:
tf.RaggedTensor.from_tensor([[1,2],[3,4]]).row_splits

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 2, 4])>

In [93]:
gnn_model.compile(loss={"policy_targets":tf.keras.losses.CategoricalCrossentropy(from_logits=False), "value_targets":"mean_squared_error"},
                           optimizer=tf.keras.optimizers.Adam())
inputs={"environment":G_s,"state":X_s}
gnn_model(inputs)

[<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[-1.],
        [-1.]], dtype=float32)>,
 <tf.RaggedTensor [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
   2.8394696e-30, 0.0, 0.0, 0.0, 0.0]                                        ,
  [3.9054356e-08, 4.65307e-11, 3.300543e-17, 1.1202969e-08, 1.0]]>]

In [89]:
inputs={"environment":G_s.to_tensor(),"state":X_s}
gnn_model(inputs)

[<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[1.],
        [1.]], dtype=float32)>,
 <tf.RaggedTensor [[0.071428575, 0.0, 0.071428575, 0.071428575, 0.071428575, 0.071428575,
   0.0, 0.071428575, 0.0, 0.071428575, 0.071428575, 0.071428575,
   0.071428575, 0.071428575, 0.071428575, 0.071428575, 0.0, 0.071428575,
   0.0, 0.0],
  [0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
   0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]>]

In [98]:
inputs={"environment":G,"state":X}
outputs={"policy_targets":P,"value_targets":V}

gnn_model.fit(inputs, outputs, batch_size=8, epochs=3, verbose=1)

Epoch 1/3
2/2 [==============================] - 3s 19ms/step - loss: 8.5328 - value_targets_loss: 1.4195 - policy_targets_loss: 7.1132
Epoch 2/3
2/2 [==============================] - 0s 19ms/step - loss: 8.2563 - value_targets_loss: 1.1973 - policy_targets_loss: 7.0591
Epoch 3/3
2/2 [==============================] - 0s 17ms/step - loss: 8.2184 - value_targets_loss: 1.2089 - policy_targets_loss: 7.0095


In [54]:
P_s.shape

TensorShape([2, None, None])

In [56]:
Z=tf.keras.layers.Input(shape=[5],dtype=tf.int32)
X=tf.keras.layers.Input(shape=[],dtype=tf.int32)

In [59]:
tf.one_hot(Z,depth=X)

2023-06-14 06:05:07.512535: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [?]
	 [[{{node Placeholder_1}}]]


<KerasTensor: shape=(None, 5, None) dtype=float32 (created by layer 'tf.one_hot_5')>

In [164]:
m=52
A2=tf.cast(tf.random.uniform([batches,m,m]) > p,dtype=tf.float32)
W2=tf.random.uniform([batches,m,m])
G2=tf.stack([A,W],axis=-1)
X2=tf.random.uniform([batches])
gnn_model.predict([G2,X2])

1/1 [==============================] - 0s 150ms/step


[array([[-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.]], dtype=float32),
 array([[0.05368359, 0.04334371, 0.04518632, 0.05064121, 0.04570146,
         0.05143947, 0.04953609, 0.04892602, 0.04699986, 0.05446874,
         0.05139836, 0.04923844, 0.05057517, 0.0522957 , 0.04983887,
         0.04774844, 0.0497077 , 0.05316909, 0.0535295 , 0.05257224],
        [0.04592943, 0.05222778, 0.05133432, 0.05049992, 0.05339243,
         0.04911728, 0.04447116, 0.04827907, 0.04652907, 0.05009769,
         0.04688105, 0.05116192, 0.05011094, 0.04744739, 0.0453846 ,
         0.04875434, 0.05631608, 0.05804124, 0.05481863, 0.04920571],
        [0.04849744, 0.04872611, 0.05237256, 0.0503167 , 0.0478346 ,
         0.05495047, 0.04880298, 0.05106444, 0.05259023, 0.048517  ,
         0.04659727, 0.05748926, 0.05129892, 0.0471

In [558]:
import pandas as pd
import networkx as nx
cora=pd.read_csv("cora/cora.content",sep="\t",header=None)
cora_graph=nx.read_edgelist("cora/cora.cites",nodetype=int)

def id_mapper(cora,cora_graph):
    mapper={}
    cora_mapped=cora.copy()
    for u,v in cora_graph.edges:
        for z in [u,v]:
            if z not in mapper:
                mapper[z]=len(mapper)
    for _,row in cora.iterrows():
        u = row[0]
        if u not in mapper:
            mapper[u]=len(mapper)
    cora_graph_mapped=nx.DiGraph()
    cora_graph_mapped.add_nodes_from(range(len(mapper)))
    for u,v in cora_graph.edges:
        if u not in mapper or v not in mapper:
            raise RuntimeErro(f"{u,v} not in mapper?")
        cora_graph_mapped.add_edge(mapper[u],mapper[v])
    cora_mapped[0]=cora_mapped[0].map(lambda x: mapper[x])
    return cora_mapped,cora_graph_mapped,mapper

    




In [559]:
cora_mapped,graph_mapped,mapper=id_mapper(cora,cora_graph)
adjacency_matrix=nx.adjacency_matrix(graph_mapped,range(len(graph_mapped.nodes)))
features_count=len(cora.columns)-1


I=range(features_count)
cora_input=cora_mapped[I]
cora_target=cora_mapped[features_count]

class_mapper=tf.keras.layers.StringLookup(output_mode="one_hot")
class_mapper.adapt(cora_target)
cora_target=class_mapper(cora_target)

In [566]:
import numpy as np
import queue
class DataLoader:
    def __init__(self,graph,inputs,targets,max_component_size):
        self.graph=graph
        inputs=inputs.to_numpy()
        S=np.zeros([len(graph),inputs.shape[-1]],dtype=np.int32)
        for row in inputs:
            S[row[0],:]=row
        Y=np.zeros([len(graph),targets.shape[-1]])
        targets=targets.numpy()
        for k,row in enumerate(inputs):
            Y[row[0],:]=targets[k,:]
        self.inputs=S
        self.targets=Y
        self.counter=0
        self.connected_components=list(nx.connected_components(self.graph.to_undirected()))
        self.max_component_size=max_component_size
        self.adjacency_matrix=nx.adjacency_matrix(self.graph,nodelist=range(len(self.graph))).todense()
    def __iter__(self):
        return self
    
    def __next__(self):
        Q=queue.Queue()
        Q.put(np.random.randint(len(self.graph)))
        S=set()
        while not Q.empty():
            u=Q.get()
            for v in self.graph.succ[u]:
                if len(S) == self.max_component_size:
                    break
                if v not in S:
                    S.add(v)
                    Q.put(v)
            if len(S) == self.max_component_size:
                break
        indexes=list(S)
        if len(indexes) < self.max_component_size:
            indexes = indexes+list(np.random.choice(len(self.graph),self.max_component_size-len(indexes)))
        self.counter+=1
        A=self.adjacency_matrix[np.ix_(indexes,indexes)]
        inputs=self.inputs[indexes,:]
        return ((A,inputs),self.targets[indexes,:])

    
    def __len__(self):
        return len(self.graph)
data_loader=DataLoader(graph_mapped,cora_input,cora_target,max_component_size=50)
signature=(  (tf.TensorSpec(shape=(None,None)),tf.TensorSpec(shape=(None,1434))),  tf.TensorSpec(shape=(None,8))  )
dataset=tf.data.Dataset.from_generator(lambda :data_loader,output_signature=signature).batch(32)
for (A0,A1),A2 in dataset:
    print(A2.shape)
    break
    
validation_dataset=tf.data.Dataset.from_generator(lambda :data_loader,output_signature=signature).batch(32).take(64)

(32, 50, 8)


2023-06-14 23:29:23.145116: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [564]:
nodes=len(mapper)
x=tf.keras.layers.Input(shape=(None,1434),name="input")
g=tf.keras.layers.Input(shape=(None,None),name="graph")
y=gnn.gcn.WGCN(24,activation="relu",name="H")((g,g,x))
y=gnn.gcn.WGCN(32,activation="relu",name="B")((g,g,y))
output=y
output=tf.keras.layers.Dense(8,activation="softmax",name="prediction")(output)
model=tf.keras.Model(inputs=[g,x],outputs=output)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer="adam",metrics=["accuracy"])

In [568]:
model.fit(dataset,epochs=12,steps_per_epoch=84,validation_data=validation_dataset)

Epoch 1/12
84/84 [==============================] - 5s 58ms/step - loss: 0.3832 - accuracy: 0.9254 - val_loss: 0.2036 - val_accuracy: 0.9667
Epoch 2/12
84/84 [==============================] - 5s 58ms/step - loss: 0.1925 - accuracy: 0.9716 - val_loss: 0.1739 - val_accuracy: 0.9775
Epoch 3/12
84/84 [==============================] - 5s 57ms/step - loss: 0.1590 - accuracy: 0.9764 - val_loss: 0.1418 - val_accuracy: 0.9827
Epoch 4/12
84/84 [==============================] - 5s 58ms/step - loss: 0.1955 - accuracy: 0.9686 - val_loss: 0.1952 - val_accuracy: 0.9714
Epoch 5/12
84/84 [==============================] - 5s 60ms/step - loss: 0.1902 - accuracy: 0.9714 - val_loss: 0.1952 - val_accuracy: 0.9735
Epoch 6/12
84/84 [==============================] - 5s 59ms/step - loss: 0.2386 - accuracy: 0.9593 - val_loss: 0.2189 - val_accuracy: 0.9642
Epoch 7/12
84/84 [==============================] - 4s 46ms/step - loss: 0.1593 - accuracy: 0.9773 - val_loss: 0.1358 - val_accuracy: 0.9816
Epoch 8/12
84

2023-06-14 22:13:23.143277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-14 22:13:23.163252: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (10,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (10,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (10,) where an element of shape (None, None) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

5416

In [374]:
for k in da

5416

InvalidIndexError: (array([2411, 2125,  824, 1762,  775,  237, 2429,  858, 1378,  323,  179,
        824, 2358, 1093,  579, 2242, 1281, 2453, 2653, 2468, 1593,  752,
       2141, 1326, 1263, 1198, 1523, 1744, 1148, 1114,  167,  730, 1103,
       1385, 2292, 2469, 2223, 1693, 1479, 1645, 1539, 2158, 1414,  539,
       1288, 2065, 2517, 1931, 1833, 1023]), slice(None, None, None))

In [588]:
tf.keras.layers.Dot(axes=(1,1))((x,x))

<KerasTensor: shape=(None, 1434, 1434) dtype=float32 (created by layer 'dot_4')>

In [689]:
R=tf.concat([tf.random.uniform([5,1,8,7])]*8,axis=1)
O=tf.random.uniform([5,1,8,7])

In [690]:
OO=tf.linalg.matmul(R,O,transpose_a=True)

In [693]:
OO[:,2,:,:]

<tf.Tensor: shape=(5, 7, 7), dtype=float32, numpy=
array([[[1.7621603 , 2.2894728 , 1.2296274 , 1.6020095 , 2.2495463 ,
         1.6082793 , 1.8895717 ],
        [2.5168889 , 3.0804372 , 2.3296845 , 3.0225103 , 3.9840174 ,
         2.5132146 , 2.8795948 ],
        [2.0061388 , 3.3219666 , 1.79297   , 2.1187174 , 3.1123812 ,
         2.1617856 , 2.6272242 ],
        [0.7515287 , 1.7849993 , 0.68653375, 0.9826304 , 1.6982837 ,
         1.2125562 , 1.8696753 ],
        [2.1512322 , 2.5824826 , 1.389887  , 2.3883047 , 3.697256  ,
         2.1450052 , 2.6422868 ],
        [1.9249227 , 2.2102168 , 1.6618285 , 2.3116577 , 2.8948588 ,
         2.09665   , 1.90902   ],
        [1.0622282 , 2.2239196 , 1.2219819 , 1.4968036 , 2.1375127 ,
         1.480628  , 1.998343  ]],

       [[2.3857453 , 1.6308819 , 2.0737286 , 1.1093215 , 1.7710793 ,
         1.9981996 , 2.5451553 ],
        [1.3533068 , 0.73741657, 1.5321262 , 0.45784006, 1.2796782 ,
         0.92517835, 1.5147383 ],
        [1.6755888 ,

In [ ]:
tf.__version__

In [555]:
import mpg.ml.model as mpg_model
import mpg.ml.model.mlp as mlp
import mpg.ml.model.gnn as gnn
import mpg.gnn.aggregator as agg
import mpg.gnn.gcn as gcn
import importlib
import tensorflow as tf
importlib.reload(mlp)
importlib.reload(gnn)
importlib.reload(gcn)
importlib.reload(agg)

saved_model=gnn.GNN(conv_layers=[2,5],masked_softmax=True)
graph=MeanPayoffGraph()
with open("model_data/test/input.txt","r") as file:
    s=int(file.readline())
    for line in file.readlines():
        u,v,w=map(int,line.split())
        graph.add_edge(u,v,w)
saved_model.compile(loss={"policy_targets":"categorical_crossentropy", "value_targets":"mean_squared_error"},
                           optimizer=tf.keras.optimizers.Adam())

2023-06-16 18:22:49.976673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?]
	 [[{{node Placeholder}}]]


In [617]:
import numpy as np


n=50

environment=np.transpose(graph.tensor_representation,[1,2,0])
shape=environment.shape[:-1]
W=np.random.uniform(size=shape)

m=environment.shape[0]
environment_pad=np.pad(environment,((0,n-m),(0,n-m),(0,0)))
permutation=np.random.permutation(m)
environment_permutation=environment[np.ix_(permutation,permutation,[0,1])]
environment_permutation.shape
permutation
inverse_permutation=np.zeros(permutation.shape)
inverse_permutation[permutation]=np.arange(m)
inverse_permutation,permutation

stack_environment=tf.ragged.stack([environment,environment_pad])
stack_states=tf.constant([s,s])

In [618]:
def get_batch(environment,starting_position):
    return {"environment":environment[None],
            "state":np.array([starting_position])}

def get_input(environment,starting_position):
    return {"environment":environment,
            "state":starting_position}

saved_model(get_input(stack_environment,stack_states))

ValueError: Exception encountered when calling layer 'tf.__operators__.getitem_485' (type SlicingOpLambda).

TypeError: object of type 'DynamicRaggedShape' has no len()


Call arguments received by layer 'tf.__operators__.getitem_485' (type SlicingOpLambda):
  • tensor=<DynamicRaggedShape lengths=[2, (5, 50), (5, 5, 5, 5, 5, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50), 2] num_row_partitions=2>
  • slice_spec=1
  • var=None

In [539]:
saved_model(get_batch(environment_permutation,permutation[s]))

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.19112538]], dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=
 array([[0.0000000e+00, 8.9281546e-21, 8.9281546e-21, 0.0000000e+00,
         1.0000000e+00]], dtype=float32)>]

In [419]:
saved_model(get_batch(environment_permutation,permutation[s]))

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.2273756]], dtype=float32)>,
 <tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[0., 0., 0., 0., 1.]], dtype=float32)>]

In [279]:
saved_model.load_weights("model_data/weights")
saved_model.predict({"environment":environment[None],"state":np.array([s])})

1/1 [==============================] - 0s 17ms/step


[array([[-0.7402104]], dtype=float32),
 array([[0.82010096, 0.00092704, 0.00092704, 0.14563765, 0.03240733]],
       dtype=float32)]

In [297]:
saved_model.save_weights("model_data/weights")

In [293]:
saved_model.save("model_data/model")


2023-06-16 06:41:02.593460: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'environment' with dtype float and shape [?,?,?,2]
	 [[{{node environment}}]]
2023-06-16 06:41:02.593541: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'state' with dtype float and shape [?]
	 [[{{node state}}]]
2023-06-16 06:41:02.649065: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'environment' with dtype float and shape [?,?,?,2]
	 [[{{node environment}}]

INFO:tensorflow:Assets written to: model_data/model/assets


INFO:tensorflow:Assets written to: model_data/model/assets


In [210]:
saved_model.get_layer("activation_168_layer_call_fn")

ValueError: No such layer: activation_168_layer_call_fn. Existing layers are: ['environment', 'state', 'tf.cast_85', 'tf.compat.v1.shape_43', 'reshape_42', 'tf.__operators__.getitem_209', 'tf.one_hot_43', 'tf.cast_86', 'tf.__operators__.getitem_210', 'tf.__operators__.getitem_211', 'tf.compat.v1.transpose_84', 'conv_0', 'conv_batch_norm_0', 'conv_1', 'conv_batch_norm_1', 'tf.compat.v1.transpose_85', 'tf.math.multiply_42', 'tf.math.reduce_sum_41', 'flat_batch_norm_0', 'policy_logits', 'flat_0', 'tf.__operators__.getitem_213', 'value_targets', 'policy_targets'].

In [236]:
saved_model.summary()

Model: "gnn_48"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 environment (InputLayer)       [(None, None, None,  0           []                               
                                 2)]                                                              
                                                                                                  
 state (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 tf.cast_101 (TFOpLambda)       (None,)              0           ['state[0][0]']                  
                                                                                                  
 tf.compat.v1.shape_51 (TFOpLam  (4,)                0           ['environment[0][0]']       

In [299]:
saved_model.to_yaml()

RuntimeError: Method `model.to_yaml()` has been removed due to security risk of arbitrary code execution. Please use `model.to_json()` instead.

In [514]:
u=tf.Variable(tf.random.uniform([8,3]),trainable=True)

M= tf.random.uniform(u.shape) > 0.1

from mpg.ml.layers import activation
importlib.reload(activation)
L=[]
with tf.GradientTape(persistent=True) as tape:
    v=activation.MaskedSoftmax()([u,M])
    w=tf.nn.softmax(u)


In [515]:
M

<tf.Tensor: shape=(8, 3), dtype=bool, numpy=
array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True, False],
       [ True,  True,  True]])>

In [516]:
tape.g(v,u)

<tf.Tensor: shape=(8, 3), dtype=float32, numpy=
array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [2.3794733e-08, 2.0254383e-08, 1.5555525e-08],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]], dtype=float32)>

In [529]:
tf.reduce_sum(tape.jacobian(w,u),axis=-2)

<tf.Tensor: shape=(8, 3, 3), dtype=float32, numpy=
array([[[ 0.20835917, -0.10748344, -0.10087571],
        [-0.10748344,  0.23128423, -0.12380078],
        [-0.10087571, -0.12380078,  0.22467649]],

       [[ 0.18941383, -0.13467373, -0.05474011],
        [-0.13467373,  0.24906918, -0.11439546],
        [-0.05474011, -0.11439546,  0.16913557]],

       [[ 0.23984125, -0.13565621, -0.10418502],
        [-0.13565621,  0.22433986, -0.08868363],
        [-0.10418502, -0.08868363,  0.19286868]],

       [[ 0.19402254, -0.08487549, -0.10914704],
        [-0.08487549,  0.21839607, -0.13352056],
        [-0.10914704, -0.13352056,  0.24266762]],

       [[ 0.16292836, -0.09776253, -0.06516585],
        [-0.09776253,  0.24947442, -0.15171191],
        [-0.06516585, -0.15171191,  0.21687774]],

       [[ 0.19045803, -0.1016817 , -0.08877633],
        [-0.1016817 ,  0.23943487, -0.13775316],
        [-0.08877633, -0.13775316,  0.22652948]],

       [[ 0.14839028, -0.07018186, -0.07820842],
      

In [475]:
u

<tf.Variable 'Variable:0' shape=(8, 3) dtype=float32, numpy=
array([[0.6968864 , 0.7271286 , 0.37946403],
       [0.56869805, 0.26584172, 0.69143724],
       [0.9783075 , 0.6419929 , 0.4552238 ],
       [0.4355247 , 0.27094483, 0.74722075],
       [0.60309887, 0.9367653 , 0.26051795],
       [0.9808819 , 0.48755777, 0.9154712 ],
       [0.02485013, 0.05360496, 0.15141165],
       [0.4767388 , 0.3148247 , 0.5395689 ]], dtype=float32)>

In [523]:
tf.test.compute_gradient(
    tf.nn.softmax, (u.value(),), delta=None
)


((array([[ 0.20835917, -0.10748344, -0.10087571,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ],
         [-0.10748344,  0.23128423, -0.12380078,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ],
         [-0.10087571, -0.12380078,  0.22467649,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
           0.        ,  0.        ,  0.        , 

In [750]:
R=tf.ragged.stack([tf.random.uniform([5,5,2]),tf.random.uniform([3,3,2])])
shape=tf.shape(R)
shape

<DynamicRaggedShape lengths=[2, (5, 3), (5, 5, 5, 5, 5, 3, 3, 3), 2] num_row_partitions=2>

In [690]:
T=tf.RaggedTensor.from_tensor(R.to_tensor(),[[5,3],[5,3]],ragged_rank=2)
S=tf.shape(R)
T==R

False

In [692]:
S.from_tensor

<bound method DynamicRaggedShape.from_tensor of <class 'tensorflow.python.ops.ragged.dynamic_ragged_shape.DynamicRaggedShape'>>

In [711]:
S[:-1]

<DynamicRaggedShape lengths=[2, (5, 3)] num_row_partitions=1>

In [714]:
S[:-1].row_partitions

(tf.RowPartition(row_splits=[0 5 8]),)

In [652]:
z.to_tensor()

<KerasTensor: shape=(None, None, None, 2) dtype=float32 (created by layer 'input.to_tensor')>

In [721]:
tf.RaggedTensor.from_nested_row_splits(R,R.nested_row_splits)

InvalidArgumentError: Arguments to _from_row_partition do not form a valid RaggedTensor
Condition x == y did not hold.
First 1 elements of x:
[34]
First 1 elements of y:
[2]

In [753]:
Z=tf.zeros([2,5])
tf.RaggedTensor.from_tensor(Z,R.row_lengths())

<tf.RaggedTensor [[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]>

In [758]:
R.row_lengths()

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 3])>

In [748]:
R.row_lengths()

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([5, 3])>

In [749]:
R.shape

TensorShape([2, None, None])

2

<KerasTensor: shape=(None,) dtype=int64 (created by layer 'input.row_lengths')>

In [775]:
R.consumers()

[]

In [776]:
tf.constant([]).con

<tf.Tensor: shape=(0,), dtype=float32, numpy=array([], dtype=float32)>

In [778]:
tf.RaggedTensor.from_tensor(tf.constant([[1],[2]]))

<tf.RaggedTensor [[1],
 [2]]>

In [824]:
z.ragged_rank

2

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'ragged_rank'

In [983]:
class Multiply(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.w=tf.Variable(tf.constant(5.),dtype=tf.float32,name="w",trainable=True)
        
    def call(self,inputs):
        return inputs * self.w


x=tf.keras.layers.Input(shape=[None])
z=tf.Variable([1.],dtype=tf.float32,trainable=True)
y=Multiply()(tf.reduce_sum(x,axis=-1))
M=tf.keras.Model(inputs=x,outputs=y)
M.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer="adam")
I=tf.ragged.constant([[0,1,3,1],[1,2,3],[5],[]],dtype=tf.float32)
J=tf.constant([0.,0,0,0])
M.fit(tf.tile(I[None],multiples=[1000,1,1]),tf.tile(J[None],multiples=[1000,1]),epochs=64,verbose=True)

Epoch 1/64


2023-06-16 23:30:26.620037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [1000]
	 [[{{node Placeholder/_0}}]]
2023-06-16 23:30:26.620473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [1000,4]
	 [[{{node Placeholder/_1}}]]


32/32 [==============================] - 1s 5ms/step - loss: 20.1128
Epoch 2/64
32/32 [==============================] - 0s 5ms/step - loss: 19.9848
Epoch 3/64
32/32 [==============================] - 0s 5ms/step - loss: 19.8568
Epoch 4/64
32/32 [==============================] - 0s 5ms/step - loss: 19.7288
Epoch 5/64
32/32 [==============================] - 0s 4ms/step - loss: 19.6008
Epoch 6/64
32/32 [==============================] - 0s 5ms/step - loss: 19.4728
Epoch 7/64
32/32 [==============================] - 0s 5ms/step - loss: 19.3448
Epoch 8/64
32/32 [==============================] - 0s 4ms/step - loss: 19.2168
Epoch 9/64
32/32 [==============================] - 0s 4ms/step - loss: 19.0889
Epoch 10/64
32/32 [==============================] - 0s 4ms/step - loss: 18.9609
Epoch 11/64
32/32 [==============================] - 0s 4ms/step - loss: 18.8329
Epoch 12/64
32/32 [==============================] - 0s 5ms/step - loss: 18.7049
Epoch 13/64
32/32 [=============================

In [980]:
M.trainable_variables

[<tf.Variable 'w:0' shape=() dtype=float32, numpy=5.0>]

In [978]:
M.trainable_variables

[<tf.Variable 'w:0' shape=() dtype=float32, numpy=0.006209164>]